# Fit example

This notebook is the most basic demonstration of running a single retrieval and plotting some relevant quantities.

In [ ]:
cd("../")

In [ ]:
# Let us observe the progress during the RT computations
ENV["XRTM_PROGRESS"] = "1"

# For good font rendering, we suggest installing the "JuliaMono" font
# which provides full support for the various Unicode glyphs.
ENV["GKS_FONTPATH"] = "~/Library/Fonts/"

In [ ]:
# Load these modules ahead of running the application
using Plots, LaTeXStrings
gr()

In [ ]:
Plots.default()

Plots.default(
    fontfamily = "JuliaMono-Regular",
    titlefont = (10, "JuliaMono-Regular"),
    legendfont = (8, "JuliaMono-Regular"),
    guidefont = (8, "JuliaMono-Regular", :black),
    tickfont = (8, "JuliaMono-Regular", :black),
    rightmargin=5Plots.mm,
    leftmargin=5Plots.mm,
    bottommargin=5Plots.mm
)

In [ ]:
# Define command-line arguments
my_args = [
    "--solar_model",  "./example_data/l2_solar_model.h5", # Path to the solar model file
    "--L1b", "./example_data/2021030111564431_inputs.h5", # Path to the L1B data location
    "--L2Met", "./example_data/2021030111564431_inputs.h5", # Path to the L2Met data location
    "--L2CPr", "./example_data/2021030111564431_inputs.h5", # Path to the L2CPr data location
    #######################################################
    "--sounding_id", "2021030111564431", # Sounding ID to retrieve
    "--spec", "1,2,3", # Which spectra to retrieve? 1 = O2-A, 2 = Weak CO2 (1.6 µm), 3 = Strong CO2 (2.06 µm)
    "--polarized", "true", # Include polarization (needed for proper OCO-2/3)?
    "--LSI", "true", # Use LSI for multiple scattering acceleration?
    "--aerosols", "true", # Include aerosols in atmosphere?
    "--retrieve_aerosols", "true", # Co-retrieve aerosol parameters? (height, width, AOD)
    "--o2_scale", "1.0048", # Spectroscopy scaling factor for Oxygen
    "--co2_scale_weak", "0.994", # Spectroscopy scaling factor for CO2 for the weak CO2 band
    "--co2_scale_strong", "0.998", # Spectroscopy scaling factor for CO2 for the strong CO2 band
    "--gamma", "100.0", # Levenberg-Marquardt γ parameter (to avoid diverging steps, we make this large)
    "--dsigma_scale", "3.0", # dσ^2 ~ a parameter that controls convergence
    "--max_iterations", "10", # Number of maximal iterations
    "--output", "2021030111564431.h5", # Name of the output file
]

# Get rid of existing command line arguments
empty!(ARGS)

# Push them in
for a in my_args
    push!(ARGS, a)
end

# Run the retrieval and return the Buffer and Solver objects! 
# (Note! This will take several minutes)
buf, solver, fm_kwargs = include("./run.jl");

In [ ]:
K = create_K_from_solver(solver);
Sa = solver.prior_covariance;
Se = create_Se_from_solver(solver);
Fx = get_modeled(solver);
y = get_measured(solver);
x = get_current_value(solver.state_vector);
xa = get_prior_value(solver.state_vector);

J_measurement = dot(y - Fx, inv(Se) * (y - Fx));
J_prior = dot(x - xa, inv(Sa) * (x - xa));

@printf "J_measurment: %.2f, J_prior: %.2f" J_measurement  J_prior

## Plot the multi-band fit as the inversion sees it

In [ ]:
# This grabs the measured and modeled radiances
measured = get_measured(solver);
modelled = get_modeled(solver);

# Plot the multi-band fit
Plots.plot(measured, size=(900, 500), linewidth=2, label="Measured", leftmargin=5Plots.mm, rightmargin=5Plots.mm);
Plots.plot!(modelled, linestyle=:dash, label="Fit")
Plots.xlabel!("Spectral sample #");
Plots.ylabel!("Radiance\n[ph μm⁻¹ m⁻² s⁻¹ sr⁻¹]");
Plots.title!("3-Band Fit")

## Closer look at the individual spectral windows and their spectral residuals

In [ ]:
# Loop through each spectral window
for swin in buf.spectral_window

    # 
    rt = buf.rt[swin];

    # For this spectral window, grab the wavelenght, measured radiance, model radiance,
    # and noise-equivalent radiances. These will the vectors of the same length that can
    # be used for plotting.
    wavelength = get_wavelength(solver, swin);
    measured = get_measured(solver, swin);
    modeled = get_modeled(solver, swin);
    noise = get_noise(solver, swin);

    # We calculate the relative residuals
    resid = @. (modeled - measured) / noise;

    # Create a two-panel plot
    
    # First plot - measured and modeled radiance
    p1 = Plots.plot(wavelength, measured, label="Measured", linewidth=2,
        leftmargin=5Plots.mm, rightmargin=5Plots.mm, bottommargin=5Plots.mm, size=(800, 500));
    Plots.plot!(wavelength, modeled, label="Fit", linestyle=:dash)
    Plots.ylabel!("Radiance\n[$(rt.radiance_unit)]")
    
    # Second plot - radiance residuals
    p2 = Plots.plot(wavelength, resid, label="Residual", linewidth=2,
        leftmargin=5Plots.mm, rightmargin=5Plots.mm, bottommargin=5Plots.mm);
    Plots.xlabel!("Wavelength [$(swin.ww_unit)]")
    Plots.ylabel!("Residual\n(fraction of noise)")
    disp_plot = plot(p1, p2, layout=(2, 1))
    display(disp_plot)
    
end

## Posterior analysis

In [ ]:
calculate_xgas(buf.scene.atmosphere)

In [ ]:
# Calculate optimal estimation-related quantities..
q = calculate_OE_quantities(solver);

In [ ]:
# Do a correlation matrix based on posterior covariance
C = similar(q.Shat);
for idx in CartesianIndices(C)
    i, j = idx.I

    C[i,j] = q.Shat[i,j] / sqrt(q.Shat[i,i] * q.Shat[j,j])

end

Plots.heatmap(C, yflip=true, clims=(-1,1), c=:RdBu_5,
    rightmargin=20Plots.mm,
    colorbar_title=" \n\nCorrelation coefficient", colorbar_titlefont=(8, "JuliaMono-Regular"))
Plots.xlabel!("State vector element")
Plots.ylabel!("State vector element")

In [ ]:
#=
    Calculate and plot the XCO2 averaging kernel (normalized)
=#

# Get the state vector indices for the CO2 profile
for atm in buf.scene.atmosphere.atm_elements
    if atm isa GasAbsorber && atm.gas_name == "CO2"
        global gas_co2 = atm
    end
end

idx = idx_for_profile_sve(gas_co2, solver.state_vector)

# Pressure weights
h = create_pressure_weights(buf.scene.atmosphere);
# Calculate the normalized averaging kernel
ak_norm = (h' * q.AK[idx,idx])' ./ h

# Plot it!
Plots.plot(
    ak_norm,
    buf.scene.atmosphere.pressure_levels,
    marker=:o,
    yflip=true,
    label=nothing,
    size=(400,400)
    )
Plots.xlabel!("Normalized Averaging Kernel")
Plots.ylabel!("Pressure level [$(buf.scene.atmosphere.pressure_unit)]")